<a href="https://colab.research.google.com/github/justinballas/WorkAutomation/blob/main/BICFR_Subaccount_Rankings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xlsxwriter

     |████████████████████████████████| 153kB 3.7MB/s 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xlsxwriter
%matplotlib inline

In [ ]:
carriers_list = ['MassMutual', 'Protective', 'Nationwide', 'Pacific Life', 'Lincoln']
carriers_list.reverse()
# ,
carrier_dfs = {}
for carrier in carriers_list:
  df = pd.read_excel('Subaccount_Rankings_data_03062021.xlsx', sheet_name=carrier)
  carrier_dfs[carrier] = df
 #skipfooter=13

for carrier, df in carrier_dfs.items():
  df.rename(columns=lambda x: x.replace('\n',''), inplace=True)

#replace_list = ["US Insurance ","US Fund ", "US VUL Sub ", "US VL Sub "]

for carrier in carriers_list:
  temp_df = carrier_dfs[carrier]
  temp_df.drop(temp_df[temp_df['Closed to New Inv']=='Yes'].index, inplace=True)
#  temp_df['Name'] = temp_df['Name'].str.split("-", 1).apply(lambda x: x[1])
#  for phrase in replace_list:
#  temp_df['Morningstar Category'] = temp_df['Morningstar Category'].str.replace(phrase, "")


mnstar_ratings = []
for carrier, df in carrier_dfs.items():
  temp_df = df
  temp_df.insert(loc=0, column='Carrier', value=carrier)
  mnstar_ratings.append(temp_df)
  print(temp_df)

ratings_df = pd.concat(mnstar_ratings)

ratings_df.drop(df[df['Closed to New Inv']=='Yes'].index, inplace=True)

agg_df_carrier = pd.pivot_table(ratings_df, index='Carrier', fill_value=np.nan)

agg_df_class = pd.pivot_table(ratings_df, index='Morningstar Category', fill_value=np.nan)

fig = plt.figure(figsize=[15, 15])



#Creating Hsitogram
for carrier in enumerate(carrier_dfs.keys()):
  temp_df = ratings_df[ratings_df['Carrier']==carrier[1]]
  temp_df = temp_df[['Morningstar Rating Overall','Carrier']]
  ax = fig.add_subplot(2, 3, carrier[0]+1)
  title = temp_df.iloc[0,1]
  plt.ylim(0,120)
  plt.xlabel('Morningstar Overall',fontsize=15)
  plt.ylabel('Frequency',fontsize=15)
  plt.title(title)
  ax.hist(temp_df['Morningstar Rating Overall'])
  plt.tight_layout()

plt.savefig('Carrier Histograms.png')


#Seperating into tables for different sheets
man_tenure = pd.merge(agg_df_carrier['Manager Tenure (Average)'], agg_df_carrier['Manager Tenure (Average)'].rank(ascending=False), how='left', left_index=True, right_index=True,suffixes=('', ' Rank'))

sharpe = pd.merge(agg_df_carrier['Sharpe Ratio 3 Yr (Mo-End) Risk Currency'], agg_df_carrier['Sharpe Ratio 3 Yr (Mo-End) Risk Currency'].rank(ascending=False), how='left', left_index=True, right_index=True,suffixes=('', ' Rank'))

exp_ratio = pd.merge(agg_df_carrier['Prospectus Net Expense Ratio'], agg_df_carrier['Prospectus Net Expense Ratio'].rank(), how='left', left_index=True, right_index=True,suffixes=('', ' Rank'))
ms_ratings_cols = ['Morningstar Rating 10 Yr', 'Morningstar Rating 5 Yr', 'Morningstar Rating 3 Yr', 'Morningstar Rating Overall']
ms_ratings =  agg_df_carrier[ms_ratings_cols]



#--------------------------------------
for col in ms_ratings_cols:
  ms_ratings["{} Rank".format(col)] = ms_ratings[col].rank(ascending=False)

#--------------------------------------
rank_cat_cols = ['Total Ret % Rank Cat 1 Yr (Mo-End)', 'Total Ret % Rank Cat 3 Yr (Mo-End)', 'Total Ret % Rank Cat 5 Yr (Mo-End)', 'Total Ret % Rank Cat 10 Yr (Mo-End)']
rank_cat = agg_df_carrier[rank_cat_cols]

for col in rank_cat_cols:
  rank_cat["{} Rank".format(col)] = rank_cat[col].rank()

#--------------------------------------
total_ret_cols = ['Total Ret 1 Yr (Mo-End) Base Currency', 'Total Ret Annlzd 3 Yr (Mo-End) Base Currency', 'Total Ret Annlzd 5 Yr (Mo-End) Base Currency', 'Total Ret Annlzd 10 Yr (Mo-End) Base Currency']
total_ret = agg_df_carrier.loc[:, total_ret_cols]

for col in total_ret_cols:
  total_ret["{} Rank".format(col)] = total_ret[col].rank(ascending=False)


#Best in class fund selection
asset_classes = ratings_df['Morningstar Category'].unique()

ratings_df['sum_ratings'] = ratings_df['Morningstar Rating 10 Yr'] + ratings_df['Morningstar Rating 5 Yr'] + ratings_df['Morningstar Rating 3 Yr'] + ratings_df['Morningstar Rating Overall']


ratings_df['avg_rankings'] = (ratings_df['Total Ret % Rank Cat 1 Yr (Mo-End)'] + ratings_df['Total Ret % Rank Cat 3 Yr (Mo-End)'] + ratings_df['Total Ret % Rank Cat 5 Yr (Mo-End)'] + ratings_df['Total Ret % Rank Cat 10 Yr (Mo-End)'])/4

ratings_df['total_score'] = ratings_df['sum_ratings'] + ratings_df['avg_rankings'].apply(lambda x: 1-(x/100))

bicf_list = []
for carrier in carriers_list:
  temp_df = ratings_df[ratings_df['Carrier']==carrier]
  for asset_class in asset_classes:
    class_df = temp_df[temp_df['Morningstar Category']==asset_class]
    class_df['BICF'] = ['yes' if i == class_df['total_score'].max() else 'no' for i in class_df['total_score']]
    bicf_list.append(class_df)
    
ratings_df = pd.concat(bicf_list)

tables_list = [ms_ratings, rank_cat, total_ret, man_tenure, exp_ratio, sharpe]

for table in tables_list:
  table = table.transpose().loc[:,['MassMutual', 'Protective', 'Nationwide', 'Pacific Life', 'Lincoln']]

# man_tenure = man_tenure.transpose()
# ms_ratings = ms_ratings.transpose()
# sharpe = sharpe.transpose()
# exp_ratio = exp_ratio.transpose()
# rank_cat = rank_cat.transpose()
# total_ret = total_ret.transpose()


#writing to excel
xlwriter = pd.ExcelWriter('BICFR Subaccount Rankings Python Output.xlsx', engine='xlsxwriter')

man_tenure.to_excel(xlwriter, sheet_name='Manager Tenure')
ms_ratings.to_excel(xlwriter, sheet_name='Ratings')
sharpe.to_excel(xlwriter, sheet_name='Sharpe')
exp_ratio.to_excel(xlwriter, sheet_name='Expense Ratio')
rank_cat.to_excel(xlwriter, sheet_name='Return Rankings')
total_ret.to_excel(xlwriter, sheet_name='Total Return')

agg_df_class.to_excel(xlwriter, sheet_name='Asset Class Breakdown')

bicf_df = ratings_df[ratings_df['BICF']=='yes']
bicf_df.set_index(['Carrier','Morningstar Category'], inplace=True)
bicf_df.to_excel(xlwriter, sheet_name='BICFR')

ratings_df.set_index(['Carrier', 'Name'], inplace=True)
ratings_df.to_excel(xlwriter, sheet_name='All Data')

empty_sheet = pd.DataFrame()
empty_sheet.to_excel(xlwriter, sheet_name='Histogram')
worksheet=xlwriter.sheets['Histogram']
worksheet.insert_image('B2','Carrier Histograms.png')

counter = 0
for i in range(len(carriers_list)):
  list(carrier_dfs.values())[i].to_excel(xlwriter, sheet_name='{}'.format(carriers_list[i]))

xlwriter.save()

xlwriter.close()

In [ ]:
agg_df_carrier

In [ ]:
carriers_list = ['MassMutual', 'Protective', 'Nationwide', 'Pacific Life', 'Lincoln']
carriers_list.reverse()
# ,
carrier_dfs = {}
for carrier in carriers_list:
  df = pd.read_excel('Subaccount_Rankings_data_03062021.xlsx', sheet_name=carrier)
  carrier_dfs[carrier] = df
 #skipfooter=13
 

In [ ]:
# mnstar_ratings = []
# for carrier, df in carrier_dfs.items():
#   temp_df = df
#   temp_df.insert(loc=0, column='Carrier', value=carrier)
#   mnstar_ratings.append(temp_df)
#   print(temp_df)

# ratings_df = pd.concat(mnstar_ratings)

agg_df_carrier = pd.pivot_table(ratings_df, index='Carrier', fill_value=np.nan)

agg_df_carrier2 = agg_df_carrier['MassMutual', 'Protective', 'Nationwide', 'Pacific Life', 'Lincoln']


In [ ]:
agg_df_carrier.transpose().loc[:,['MassMutual', 'Protective', 'Nationwide', 'Pacific Life', 'Lincoln']]